In [14]:
import os

In [15]:
%pwd

'c:\\DANE\\Git Repos\\Kidney-Disease-Classification'

In [3]:
os.chdir("../")

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: tuple

In [17]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        base_model = self.config.base_model
        params = self.params

        create_directories(training.root_dir)

        return TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(base_model.updated_base_model_path),
            training_data=Path(self.config.data_ingestion.unzip_dir),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=tuple(params.IMAGE_SIZE),
        )

In [19]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [20]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        img_size = self.config.params_image_size[:-1]  # (224,224)
        batch_size = self.config.params_batch_size

        # --- train dataset ---
        self.train_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            labels="inferred",
            label_mode="categorical",  # softmax
            batch_size=batch_size,
            image_size=img_size,
            shuffle=True,
            validation_split=0.2,
            subset="training",
            seed=42,
        )

        # --- validation dataset ---
        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            labels="inferred",
            label_mode="categorical",
            batch_size=batch_size,
            image_size=img_size,
            shuffle=False,
            validation_split=0.2,
            subset="validation",
            seed=42,
        )

        # --- normalization ---
        normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

        self.train_generator = self.train_generator.map(
            lambda x, y: (normalization_layer(x), y),
            num_parallel_calls=tf.data.AUTOTUNE
        )

        self.valid_generator = self.valid_generator.map(
            lambda x, y: (normalization_layer(x), y),
            num_parallel_calls=tf.data.AUTOTUNE,
        )

        # --- augmentation ---
        if self.config.params_is_augmentation:
            data_augmentation = tf.keras.Sequential(
                [
                    tf.keras.layers.RandomFlip("horizontal"),
                    tf.keras.layers.RandomRotation(0.1),
                    tf.keras.layers.RandomZoom(0.2),
                    tf.keras.layers.RandomTranslation(0.1, 0.1)
                ]
            )

        self.train_generator = self.train_generator.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=tf.data.AUTOTUNE,
        )

        # --- pipeline optimalization ---
        self.train_generator = self.train_generator.prefetch(
            buffer_size=tf.data.AUTOTUNE
        )
        self.valid_generator = self.valid_generator.prefetch(
            buffer_size=tf.data.AUTOTUNE
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator
        )  

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )      

In [24]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise

[2025-08-26 14:36:50,695 INFO - common : yaml file: config\config.yaml loaded successfully]
[2025-08-26 14:36:50,706 INFO - common : yaml file: params.yaml loaded successfully]
[2025-08-26 14:36:50,707 INFO - common : Created directory at: artifacts/training]
Found 7360 files belonging to 2 classes.
Using 5888 files for training.
Found 7360 files belonging to 2 classes.
Using 1472 files for validation.
Epoch 1/5
368/368 ━━━━━━━━━━━━━━━━━━━━ 328s 890ms/step - accuracy: 0.6374 - loss: 9.7409 - val_accuracy: 0.0000e+00 - val_loss: 38.4858
Epoch 2/5
368/368 ━━━━━━━━━━━━━━━━━━━━ 324s 881ms/step - accuracy: 0.7298 - loss: 5.0232 - val_accuracy: 0.9212 - val_loss: 0.5207
Epoch 3/5
368/368 ━━━━━━━━━━━━━━━━━━━━ 326s 886ms/step - accuracy: 0.7772 - loss: 3.6550 - val_accuracy: 0.9049 - val_loss: 0.7702
Epoch 4/5
368/368 ━━━━━━━━━━━━━━━━━━━━ 328s 891ms/step - accuracy: 0.8045 - loss: 2.8847 - val_accuracy: 0.9973 - val_loss: 0.0047
Epoch 5/5
368/368 ━━━━━━━━━━━━━━━━━━━━ 328s 889ms/step - accuracy